##### Import Modules

In [1]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.metrics import confusion_matrix

from sklearn.pipeline import Pipeline

from sklearn import preprocessing
from sklearn.decomposition import PCA, sparse_encode

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

import numpy as np
import scipy.io as sio

import warnings

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib notebook

##### Load Data

In [2]:
# DataUse = sio.loadmat('AllSubs_feats_reprocessed.mat')
DataUse = sio.loadmat('Z:\Lab Member Folders\Blair Hu\Git Sandbox\Aim-1-Bilateral-Exoskeleton-Intent-Recognition\Data\AllSubs_feats_reprocessed.mat')
phaseinc = 0 # Specifies whether data from both legs are merged (0 = not merged, 2= merged)

In [3]:
# The feats element has 5 columns, corresponding to delays of 0, 30, 60, 90, 120 ms
filestr = '0Delay' # For saved filename
feats_all = np.array(DataUse['feats_combined'][0,0]) # The column specifies which delay to use
legphases_all = np.array(DataUse['legphase_combined'])
trig_all = np.array(DataUse['trig_combined'])
subject_all = np.array(DataUse['subject_combined'])

# Get the entering and leaving modes from the triggers
mode_all = np.empty(trig_all.shape)
mode_leave = np.empty(trig_all.shape)
for i in range(trig_all.shape[0]):
    mode_all[i] = int(trig_all[i][0][0][2])
    mode_leave[i] = int(trig_all[i][0][0][0])

# Convert subject codes to numbers (1-10)
subjectnum_all = np.empty(subject_all.shape)
for i in range(subject_all.shape[0]):
    if str(subject_all[i][0][0]) == 'AB156':
        subjectnum_all[i] = 1
    elif str(subject_all[i][0][0]) == 'AB185':
        subjectnum_all[i] = 2
    elif str(subject_all[i][0][0]) == 'AB186':
        subjectnum_all[i] = 3
    elif str(subject_all[i][0][0]) == 'AB188':
        subjectnum_all[i] = 4
    elif str(subject_all[i][0][0]) == 'AB189':
        subjectnum_all[i] = 5
    elif str(subject_all[i][0][0]) == 'AB190':
        subjectnum_all[i] = 6
    elif str(subject_all[i][0][0]) == 'AB191':
        subjectnum_all[i] = 7
    elif str(subject_all[i][0][0]) == 'AB192':
        subjectnum_all[i] = 8
    elif str(subject_all[i][0][0]) == 'AB193':
        subjectnum_all[i] = 9
    elif str(subject_all[i][0][0]) == 'AB194':
        subjectnum_all[i] = 10

##### Remove non-walking trials

In [4]:
# Remove non-walking gait events
keepinds = np.asarray([i for i in np.arange(len(mode_all)) if ((0 < mode_all[i] < 6) & (0 < mode_leave[i] < 6))])

feats_all = feats_all[keepinds,:]
legphases_all = legphases_all[keepinds]
subject_all = subject_all[keepinds]
trig_all = trig_all[keepinds]
subjectnum_all = subjectnum_all[keepinds]
mode_all = mode_all[keepinds]
mode_leave = mode_leave[keepinds]

# Encode class as one-hot for ANN
onehotmode = np.zeros((mode_all.shape[0],5))
for i in range(mode_all.shape[0]):
    onehotmode[i,int(mode_all[i])-1] = 1

#### Calculate the number of gait events for each leg for each subject (after removal of standing transitions)

In [5]:
subjsteps = np.empty((10,4))
for phaseind in range(1,5):
    for subjind in range(1,11):
        phase = np.where(legphases_all == phaseind)[0]
        subj = np.where(subjectnum_all == subjind)[0]
        subjsteps[subjind-1,phaseind-1] = len(np.intersect1d(phase,subj))

# Heel contact
print('HC:')
print(np.mean(np.hstack((subjsteps[:,0],subjsteps[:,2]))))
print(np.std(np.hstack((subjsteps[:,0],subjsteps[:,2]))))

# Toe off
print('TO:')
print(np.mean(np.hstack((subjsteps[:,1],subjsteps[:,3]))))
print(np.std(np.hstack((subjsteps[:,1],subjsteps[:,3]))))

HC:
516.55
47.4904990498
TO:
522.65
48.5296558817


##### Define feature sets

In [6]:
# Make feature sets from labels
featlabels = DataUse['featlabels']
imufeats = np.arange(0,180)
goniofeats = np.arange(180,228)
emgfeats = np.arange(228,368)
ipsifeats = []
contrafeats = []
waistfeats = []
LLemgfeats = []
ULemgfeats = []
shankfeats = []
thighfeats = []
kneefeats = []
anklefeats = []

LLemg = ['TA','MG','SOL']
ULemg = ['BF','ST','RF','VL']

for i in range(featlabels.shape[0]):
    if 'Ipsi' in str(featlabels[i]):
        ipsifeats.append(i)
    if 'Contra' in str(featlabels[i]):
        contrafeats.append(i)
    if  'Waist' in str(featlabels[i]):
        waistfeats.append(i)
    if any(muscles in str(featlabels[i]) for muscles in LLemg):
        LLemgfeats.append(i)
    if any(muscles in str(featlabels[i]) for muscles in ULemg):
        ULemgfeats.append(i)
    if 'Shank' in str(featlabels[i]):
        shankfeats.append(i)
    if 'Thigh' in str(featlabels[i]):
        thighfeats.append(i)
    if 'Knee' in str(featlabels[i]):
        kneefeats.append(i)
    if 'Ankle' in str(featlabels[i]):
        anklefeats.append(i)

ipsifeats = np.array(ipsifeats)
contrafeats = np.array(contrafeats)
waistfeats = np.array(waistfeats)

ipsi_emg = np.intersect1d(emgfeats,ipsifeats)
ipsi_gonio = np.intersect1d(goniofeats,ipsifeats)
ipsi_imu = np.intersect1d(imufeats,ipsifeats)
ipsi_all = ipsifeats

ipsi_emg_gonio = np.union1d(ipsi_emg,ipsi_gonio)
ipsi_emg_imu = np.union1d(ipsi_emg,ipsi_imu)
ipsi_gonio_imu = np.union1d(ipsi_gonio,ipsi_imu)

contra_emg = np.intersect1d(emgfeats,contrafeats)
contra_gonio = np.intersect1d(goniofeats,contrafeats)
contra_imu = np.intersect1d(imufeats,contrafeats)
contra_all = contrafeats

ipsi_all_contra_emg = np.union1d(ipsi_all,contra_emg)
ipsi_all_contra_gonio = np.union1d(ipsi_all,contra_gonio)
ipsi_all_contra_imu = np.union1d(ipsi_all,contra_imu)

ipsi_all_contra_emg_gonio = np.union1d(ipsi_all_contra_emg,contra_gonio)
ipsi_all_contra_emg_imu = np.union1d(ipsi_all_contra_emg,contra_imu)
ipsi_all_contra_gonio_imu = np.union1d(ipsi_all_contra_gonio,contra_imu)

bilat_emg = emgfeats
bilat_gonio = goniofeats
bilat_imu = np.union1d(ipsi_imu,contra_imu)
bilat_all = np.union1d(np.union1d(bilat_emg,bilat_gonio),bilat_imu)

# Added these sensor sets for full comparison
ipsi_LLemg = np.intersect1d(LLemgfeats,ipsifeats) 
ipsi_ULemg = np.intersect1d(ULemgfeats,ipsifeats) 
contra_LLemg = np.intersect1d(LLemgfeats,contrafeats) 
contra_ULemg = np.intersect1d(ULemgfeats,contrafeats) 
ipsi_emg_contra_LLemg = np.union1d(ipsi_emg,contra_LLemg) 
ipsi_emg_contra_ULemg = np.union1d(ipsi_emg,contra_ULemg) 

ipsi_knee = np.intersect1d(kneefeats,ipsifeats) 
ipsi_ankle = np.intersect1d(anklefeats,ipsifeats) 
contra_knee = np.intersect1d(kneefeats,contrafeats) 
contra_ankle = np.intersect1d(anklefeats,contrafeats) 
ipsi_gonio_contra_knee = np.union1d(ipsi_gonio,contra_knee)  
ipsi_gonio_contra_ankle = np.union1d(ipsi_gonio,contra_ankle) 

ipsi_shank = np.intersect1d(shankfeats,ipsifeats) 
ipsi_thigh = np.intersect1d(thighfeats,ipsifeats) 
contra_shank = np.intersect1d(shankfeats,contrafeats) 
contra_thigh = np.intersect1d(thighfeats,contrafeats)
ipsi_imu_contra_shank = np.union1d(ipsi_imu,contra_shank) 
ipsi_imu_contra_thigh = np.union1d(ipsi_imu,contra_thigh) 

featsdict = {'1': ipsi_emg, '2': ipsi_gonio, '3': ipsi_imu, '4': ipsi_emg_gonio, '5': ipsi_emg_imu, '6': ipsi_gonio_imu,
             '7': ipsi_all, '8': ipsi_all_contra_emg, '9': ipsi_all_contra_gonio, '10': ipsi_all_contra_imu, 
             '11': ipsi_all_contra_emg_gonio, '12': ipsi_all_contra_emg_imu, '13': ipsi_all_contra_gonio_imu,
             '14': bilat_emg, '15': bilat_gonio, '16': bilat_imu, '17': bilat_all, '18': ipsi_LLemg, '19': ipsi_ULemg, 
             '20': ipsi_emg_contra_LLemg, '21': ipsi_emg_contra_ULemg, '22': ipsi_knee, '23': ipsi_ankle, '24': ipsi_gonio_contra_knee, '25': ipsi_gonio_contra_ankle,
             '26': ipsi_shank, '27': ipsi_thigh, '28': ipsi_imu_contra_shank, '29': ipsi_imu_contra_thigh, '30': contra_emg, '31': contra_gonio, '32': contra_imu, '33': contra_all}

featskeys = ('1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33')
sensorsets = len(featskeys)

##### Define Scale, Dimensionality Reduction, Pipelines, CV, and Classifiers

In [7]:
pca = PCA()
lda = LinearDiscriminantAnalysis()
svm = SVC(kernel = 'linear', C = 10)
ann = MLPClassifier(solver = 'sgd', hidden_layer_sizes = (10,), max_iter = 1000, learning_rate_init = .1, learning_rate = 'adaptive', momentum = 0.9)

scale = preprocessing.StandardScaler()
scale_PCA = Pipeline([('norm',scale),('dimred',pca)])

# Define cross-validation parameters
numfolds = 10
kf = KFold(n_splits = numfolds, shuffle = True)
skf = StratifiedKFold(n_splits = numfolds, shuffle = True)
loo = LeaveOneOut()

##### Within Subjects Analysis

In [ ]:
subjkeys = ('AB156','AB185','AB186','AB188','AB189','AB190','AB191','AB192','AB193','AB194')
legphasekeys = ('RHC','RTO','LHC','LTO')

for subnum in np.arange(1,11): # Iterate across subjects
    print('Subject: ' + subjkeys[subnum-1])

    withinresultsdict = {}
    withinresultsdict[subjkeys[subnum-1]] = {'RHC':[],'RTO':[],'LHC':[],'LTO':[]}
        
    # Get indices corresponding to subnum
    subjectspecific = np.asarray([i for i in range(len(subjectnum_all)) if subjectnum_all[i] == subnum])  
        
    # If phaseinc = 0, then each leg has its own classifier for each gait event; legphaseID should be 1,2,3,4
    # If phaseinc = 2, then each gait event has its own classifier (legs combined); legphaseID should be 1,2
    for legphaseID in np.array([1,2,3,4]): # Iterate across HC and TO
        print('Leg Phase ID: ' + legphasekeys[legphaseID-1])
                
        phasespecific = np.asarray([j for j in range(len(legphases_all)) if ((legphases_all[j] == legphaseID) or (legphases_all[j] == (legphaseID + phaseinc)))])    
        subjectlegphases = np.intersect1d(subjectspecific,phasespecific)

        # Get the data corresponding to the gait phase        
        legphasefeats_all = feats_all[subjectlegphases,:]
        legphasemodeleave_all = mode_leave[subjectlegphases]
        legphasemode_all = mode_all[subjectlegphases]
        legphaseonehotmode_all = onehotmode[subjectlegphases,:]
        legphasetrig_all = trig_all[subjectlegphases]      
        
        numtemp = np.shape(legphasemode_all)[0]
        
        # Last column for true mode
        pred_temp = np.empty((numtemp,4*sensorsets+1))

        for modeclassifier in np.array([1,2,3,4,5]): # Iterate across LW, RA, RD, SA, SD           
            ms_inds = np.where(legphasemodeleave_all == modeclassifier)[0]
            ms_feats = legphasefeats_all[ms_inds,:]
            ms_mode = legphasemode_all[ms_inds]
            ms_onehotmode = legphaseonehotmode_all[ms_inds,:]
            ms_trig = legphasetrig_all[ms_inds]

            print
            print('Mode: ' + str(modeclassifier))
            print('Unique: ' + str(np.unique(ms_mode)))
            print np.histogram(ms_mode,bins=[0, 1.5, 2.5, 3.5, 4.5, 5.5])[0]            
            
            print('Instances: ' + str(len(ms_inds)))
            
            foldcounter = 0
            for train_index, test_index in kf.split(ms_feats):    
#             for train_index, test_index in skf.split(ms_feats,ms_mode.ravel()): 
#             for train_index, test_index in loo.split(ms_feats):
                foldcounter += 1
                print('Fold: ' + str(foldcounter))

                # Make training and testing sets
                feats_train, mode_train, onehotmode_train, trig_train = ms_feats[train_index,:], ms_mode[train_index], ms_onehotmode[train_index,:], ms_trig[train_index]
                feats_test, mode_test, onehotmode_test, trig_test = ms_feats[test_index,:], ms_mode[test_index], ms_onehotmode[test_index,:], ms_trig[test_index]     

                # Iterate across all sensor sets
                for keyind in np.arange(sensorsets):
                    featskeystemp = featskeys[keyind]
                    usefeats = featsdict[featskeystemp]
                    
                    scale.fit(feats_train[:,usefeats])
                    scale_PCA.fit(feats_train[:,usefeats])
                    
                    feats_train_norm = scale.transform(feats_train[:,usefeats])
                    feats_train_PCA = scale_PCA.transform(feats_train[:,usefeats])

                    feats_test_norm = scale.transform(feats_test[:,usefeats])
                    feats_test_PCA = scale_PCA.transform(feats_test[:,usefeats])           
                    
                    pcaexplainedvar = np.cumsum(scale_PCA.named_steps['dimred'].explained_variance_ratio_)                
                    pcanumcomps = min(min(np.where(pcaexplainedvar > 0.95))) + 1
                    
                    unique_modes = np.unique(mode_train)

                    lda.set_params(priors = np.ones(len(unique_modes))/len(unique_modes))

                    pcaldafit = lda.fit(feats_train_PCA[:,0:pcanumcomps],mode_train.ravel())
                    pred_temp[ms_inds[test_index],keyind] = pcaldafit.predict(feats_test_PCA[:,0:pcanumcomps]).ravel()

                    #####
                    # Use dimensionality reduced feature set for SVM and ANN
                    #####
                    
                    # svmfit = svm.fit(feats_train_PCA[:,0:pcanumcomps],mode_train)
                    # pred_temp[ms_inds[test_index],int(featskeystemp)-1+17] = svmfit.predict(feats_test_PCA[:,0:pcanumcomps]).ravel()

                    # annfit = ann.fit(feats_train_PCA[:,0:pcanumcomps],onehotmode_train)            
                    # pred_temp[ms_inds[test_index],int(featskeystemp)-1+34] = (np.argmax(annfit.predict(feats_test_PCA[:,0:pcanumcomps]),axis=1)+1).ravel()

                    svmfit = svm.fit(feats_train_norm,mode_train.ravel())
                    pred_temp[ms_inds[test_index],keyind+sensorsets] = svmfit.predict(feats_test_norm).ravel()

                    annfit = ann.fit(feats_train_norm,onehotmode_train)      
                    pred_temp[ms_inds[test_index],keyind+2*sensorsets] = (np.argmax(annfit.predict(feats_test_norm),axis=1)+1).ravel()

                    ########################################
                    # Sparse representation classification #
                    ########################################
                    # L2-normalize features
                    feats_train_l2norm = preprocessing.normalize(feats_train_PCA[:,0:pcanumcomps],norm='l2',axis=1)
                    feats_test_l2norm = preprocessing.normalize(feats_test_PCA[:,0:pcanumcomps],norm='l2',axis=1)
                    
                    dictionary = feats_train_l2norm
                    
                    # Get binary class masks (only 1 for dictionary elements corresponding to that class)
                    class_matrix = {}
                    for label in np.unique(mode_train):
                        label_inds = np.asarray([ind for ind in np.arange(len(train_index)) if mode_train[ind] == label])
                        label_mask = np.zeros(np.shape(feats_train_l2norm))
                        if len(label_inds) > 0:
                            label_mask[label_inds,:] = 1
                            class_matrix[str(label)] = np.multiply(dictionary,label_mask)
                        else:
                            class_matrix[str(label)] = np.multiply(dictionary,label_mask)
                    
                    # Perform classification based on class with lowest reconstruction error
                    src_pred = np.empty((len(test_index),1))
                    for pred_inds in np.arange(len(test_index)):
                        feats_pred = feats_test_l2norm[pred_inds,:]
                        code = sparse_encode(X=feats_pred.reshape(1,-1),dictionary=dictionary,algorithm='omp',alpha=1)
                        resid = []
                        for label in np.unique(mode_train):
                            resid.append(np.linalg.norm(np.subtract(np.dot(code,class_matrix[str(label)]),feats_pred),ord=2))
                        src_pred[pred_inds] = np.unique(mode_train)[resid.index(min(resid))]

                    pred_temp[ms_inds[test_index],keyind+3*sensorsets] = src_pred.ravel()
                    ########################################
                
        pred_temp[:,4*sensorsets] = legphasemode_all.ravel()

        # Save predictions and triggers for all sensor sets
        withinresultsdict[subjkeys[subnum-1]][legphasekeys[legphaseID-1]] = {'Pred':[],'Trig':[]}        
        withinresultsdict[subjkeys[subnum-1]][legphasekeys[legphaseID-1]]['Pred'] = pred_temp
        withinresultsdict[subjkeys[subnum-1]][legphasekeys[legphaseID-1]]['Trig'] = legphasetrig_all
            
        print    
        print('...Finished ' + subjkeys[subnum-1] + ' ' + legphasekeys[legphaseID-1] + '...')
        
    # Save predictions and triggers for the subnum
#     sio.savemat(subjkeys[subnum-1] + '_WithinSubjectResults_' + filestr + '_ModeSpecific_AllSensorsClassifiers_LOO',withinresultsdict)
    sio.savemat(subjkeys[subnum-1] + '_WithinSubjectResults_' + filestr + '_ModeSpecific_AllSensorsClassifiers_' + str(numfolds) + 'Fold_Reprocessed',withinresultsdict)
    
    print('...File saved...')    
    print

##### Check the results of the classifiers

In [ ]:
cm = confusion_matrix(withinresultsdict['AB186']['RHC']['Pred'][:,132],withinresultsdict['AB186']['RHC']['Pred'][:,15])
print cm
print np.double(np.sum(cm.diagonal()))/np.sum(np.sum(cm))